In [1]:
import random
import pickle

import numpy as np
import pandas as pd
from nltk.tokenize import RegexpTokenizer

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
text4000 = pd.read_csv(".//Data/4000/4000-stories-VAD.csv")

In [3]:
texts= list(text4000.story.values)

In [4]:
# text_limiter=1000000 #10000 zeichen
joined_text = " ".join(texts)
# limited_text = joined_text[:text_limiter]

In [5]:
# Tokenizer erstellen und Texte darauf anwenden
tokenizer = RegexpTokenizer(r"\w+")
tokens = tokenizer.tokenize(joined_text.lower())

In [6]:
context_words = 10
input_words = []
next_words = []
words_limiter=100000 #limitiert die anzahl an Trainingsdaten
for i in range(len(tokens) - context_words):
    input_words.append(tokens[i:i + context_words])
    next_words.append(tokens[i + context_words])
    if(len(next_words)>=words_limiter):break
len(next_words)

100000

In [7]:
abc=100
print(input_words[abc])
print(next_words[abc])

['keyboard', 'and', 'very', 'large', 'sized', 'dogs', 'shall', 'romp', 'through', 'every']
page


In [8]:
input_tokens = [token for sublist in input_words for token in sublist]
unique_tokens = np.unique(np.concatenate((input_tokens, next_words))) #alle Token 108282 || in 50000 Token sind 7359 unique
unique_token_index = {token: idx for idx, token in enumerate(unique_tokens)}

In [9]:
len(unique_tokens)

9803

In [10]:
X = np.zeros((len(input_words), context_words, len(unique_tokens)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)

In [11]:
for i, words in enumerate(input_words):
    for j, word in enumerate(words):
        X[i,j, unique_token_index[word]] = 1
    Y[i, unique_token_index[next_words[i]]] = 1

In [12]:
# import itertools

# train_words = input_words
# train_words.append(next_words)
# joined_train_words=" ".join(list(itertools.chain.from_iterable(train_words)))

In [13]:
from tensorflow.keras.preprocessing.text import Tokenizer

text = " ".join(tokens[:100000])

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

word_index = tokenizer.word_index
total_unique_words = len(tokenizer.word_index) + 1   
print(total_unique_words)

9804


In [14]:
input_sequences = tokenizer.texts_to_sequences(input_words)
next_sequences = tokenizer.texts_to_sequences(next_words)

print("Input Sequences:")
print(input_sequences[1])  # Beispiel für die Umwandlung des zweiten input_words

print("Next Sequences:")
print(next_sequences[1])  # Beispiel für die Umwandlung des zweiten target_word


Input Sequences:
[10, 51, 13, 73, 29, 52, 269, 1846, 73, 29]
Next Sequences:
[526]


In [15]:
embedding_dim = 100  # Dimension der GloVe-Vektoren
embeddings_index = {}  # Dictionary für die GloVe-Vektoren

# Laden der GloVe-Daten
path = 'glove.6B/glove.6B.100d.txt'
with open(path, encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coeffs = np.array(values[1:], dtype=np.float32)
        embeddings_index[word] = coeffs

# Erstellen der embeddings_matrix
# num_words = min(words_limiter, len(unique_tokens))  # Anzahl der eindeutigen Tokens, die verwendet werden
# embeddings_matrix = np.zeros((num_words, embedding_dim))  # Initialisierung der Matrix mit Nullen

# for word, i in unique_token_index.items():
#     if i >= words_limiter:
#         continue  # Nur die ersten words_limiter eindeutigen Tokens verwenden

#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embeddings_matrix[i] = embedding_vector
#     else:
#         # Wenn das Wort nicht in den GloVe-Vektoren vorhanden ist, wird es mit zufälligen Werten initialisiert
#         embeddings_matrix[i] = np.random.uniform(-0.25, 0.25, embedding_dim)
embeddings_matrix = np.zeros((total_unique_words, 100))
for word, i in word_index.items():
   embedding_vector = embeddings_index.get(word)
   if embedding_vector is not None:
     embeddings_matrix[i] = embedding_vector;


In [16]:
# input_shape=(context_words, len(unique_tokens))

In [17]:
model = Sequential()
model.add(Embedding(input_dim = total_unique_words, output_dim=100, weights=[embeddings_matrix], input_length=10, trainable=False))
model.add(LSTM(128, recurrent_dropout=0.5, dropout=0.5,return_sequences=True))
model.add(LSTM(128, recurrent_dropout=0.5, dropout=0.5))
model.add(Dense(total_unique_words, activation="softmax"))

In [18]:
split_index = int(len(input_sequences) * 0.95)
x_train, x_val = input_sequences[:split_index], input_sequences[split_index:]
y_train, y_val = next_sequences[:split_index], next_sequences[split_index:]

In [19]:
print("Form der Eingabedaten (X):", X.shape)
print("Form der Ausgabedaten (Y):", Y.shape)

Form der Eingabedaten (X): (100000, 10, 9803)
Form der Ausgabedaten (Y): (100000, 9803)


In [20]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01), metrics=['accuracy'])
history = model.fit(x_train, y_train, validation_data=(x_val, y_val), batch_size=128, epochs=1, shuffle=True).history

 42/743 [>.............................] - ETA: 47s - loss: 7.2634 - accuracy: 0.0543

KeyboardInterrupt: 

In [ ]:
model.save('E_Model/Model.h5')
pickle.dump(history, open("Model2/history.p", "wb"))

In [ ]:
with open('Model2/MetaData/context_words.pickle', 'wb') as file:
    pickle.dump(context_words, file)
with open('Model2/MetaData/unique_tokens.pickle', 'wb') as file:
    pickle.dump(unique_tokens, file)
with open('Model2/MetaData/unique_token_index.pickle', 'wb') as file:
    pickle.dump(unique_token_index, file)   

In [ ]:
from matplotlib import pyplot as plt
model = load_model('Model2/Model.h5')
history = pickle.load(open("Model2/history.p", "rb"))

plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left') 

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')